In [1]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Correctly load the data from CSV
df = pd.read_csv("CleanedData/players_all.csv")  # Make sure the path is correct

# Function to draw a figure 
def drawFigure():
    # Example: change sepal_width and sepal_length to your actual columns
    fig = px.bar(df, x="player", y="goals", color="assists")  # Change as needed based on your data columns
    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        showlegend=True
    )
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(figure=fig, config={'displayModeBar': False})
            ])
        ),
    ])

# Text field
def drawText(text="Placeholder Text"):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2(text),
                ], style={'textAlign': 'center'})
            ])
        ),
    ])

# Build App
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col(drawText("Select Players"), width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='player-dropdown',
                        options=[{'label': i, 'value': i} for i in df['player'].unique()],  # Assuming 'player' is the column name
                        multi=True,
                        placeholder="Select players",
                        style={"color": "#000"}
                    )
                ], width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col(dcc.Graph(id='stacked-bar-chart'), width=12),
            ], align='center'),
        ]), color='dark'
    )
])

@app.callback(
    Output('stacked-bar-chart', 'figure'),
    [Input('player-dropdown', 'value')]
)
def update_graph(selected_players):
    print("Callback triggered with selected players:", selected_players)
    if not selected_players:
        print("No players selected. Returning empty figure.")
        return go.Figure()

    fig = go.Figure()
    for player in selected_players:
        filtered_df = df[df['player'] == player]
        print(f"Processing data for player: {player}")
        goals_sum = filtered_df['goals'].sum()
        assists_sum = filtered_df['assists'].sum()
        
        fig.add_trace(go.Bar(name='Goals', x=[player], y=[goals_sum], marker_color='indigo'))
        fig.add_trace(go.Bar(name='Assists', x=[player], y=[assists_sum], marker_color='green'))
    
    fig.update_layout(
        barmode='stack',
        title='Player Goals and Assists',
        xaxis_title='Player',
        yaxis_title='Count',
        legend_title='Metric'
    )
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
